### Name: Arash Asgari
### Student number: 400201037

---

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances

In [5]:
def create_fake_data(N, d):
    # returns X array with shape (N, d)
    vec = np.random.rand(N, d)
    vec = np.floor(vec * 99)
    return vec

def cosine_distance(x, y):
    # returns cosine distance between x and y
    cos_sim = np.dot(x, y.T)/(np.linalg.norm(x)*np.linalg.norm(y))
    return 1 - cos_sim

def find_k_nearest_neighbours(X, q, k):
    # returns indexes of the k-nearest-neighbours of vector q
    distances = np.zeros(X.shape[0])
    for ind, vec in enumerate(X):
      distances[ind] = cosine_distance(vec, np.array(q))
    return np.argsort(distances)[:k]

def find_k_farest_neighbours(X, q, k):
    # returns indexes of the k-farest-neighbours of vector q
    distances = np.zeros(X.shape[0])
    for ind, vec in enumerate(X):
      distances[ind] = cosine_distance(vec, q)
    return np.argsort(distances)[-k:]

      

In [6]:
f_data = create_fake_data(1000, 2)
data = np.array([[3, 4], [5, 10], [-3, -4], [6, 9]])
find_k_nearest_neighbours(data, (3, 4), 3)

array([0, 3, 1])

Each object of hash table is a min hashing instance.

In [7]:
class HashTable:
    def __init__(self, hash_size, inp_dimensions):
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_table = dict()
        self.projections = np.random.randn(self.hash_size, inp_dimensions)
        
    def generate_hash(self, inp_vector):
        bools = (np.dot(inp_vector, self.projections.T) > 0).astype('int')
        bools = np.squeeze(bools)
        return ''.join(bools.astype('str'))

    def __setitem__(self, inp_vec, label):
        hash_value = self.generate_hash(inp_vec)
        self.hash_table[hash_value] = self.hash_table\
            .get(hash_value, list()) + [label]
        
    def __getitem__(self, inp_vec):
        hash_value = self.generate_hash(inp_vec)
        return self.hash_table.get(hash_value, [])
        
hash_table = HashTable(hash_size=4, inp_dimensions=20)

creates 'num_tables' hash tables of size specified by 'hash_size'. the inp_dimension keeps the number of input features. Afterward, finds if two vectors are the same using at least one hashtable.

In [8]:
class LSH:
    def __init__(self, num_tables, hash_size, inp_dimensions):
        self.num_tables = num_tables
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_tables = list()
        for i in range(self.num_tables):
            self.hash_tables.append(HashTable(self.hash_size, self.inp_dimensions))
    
    def __setitem__(self, inp_vec, label):
        for table in self.hash_tables:
            table[inp_vec] = label
    
    def __getitem__(self, inp_vec):
        results = list()
        for table in self.hash_tables:
            results.extend(table[inp_vec])
        return list(set(results))
    
    def generate_projection(self, inp_vec):
      projections = [ hash_table.generate_hash(inp_vec) for hash_table in self.hash_tables]
      return projections

    def check_similarity(self, point_projections, in_vec_projections):
      for point_table_projection, point_vec_projection in zip(point_projections, in_vec_projections):
        if point_table_projection == point_vec_projection:
          return True
      return False
    
    def query(self, points, query_point):
      in_vec_projections = self.generate_projection(query_point)
      results = []
      for ind, point in enumerate(points):
        point_projections = self.generate_projection(point)
        if self.check_similarity(point_projections, in_vec_projections):
          results.append(ind)
      return results
        

Here, we generated 1000 random vectors with the dimensions of 300 and used 20 hash tables hashing each vector to another vector of length 30. 

In [22]:
N = 1000
d = 300
f = 100
b = 20
r = 30
lsh = LSH(b, r, d)
points = create_fake_data(N, d)
in_vec =  create_fake_data(1, d)
results_lsh = lsh.query(points ,in_vec)
results_k_farest = find_k_farest_neighbours(points, in_vec, 100)